In [26]:
import os
from langchain.llms import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv('file.env'))

True

# Tracking token usage

In [27]:
from langchain.callbacks import get_openai_callback

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", n=2, best_of=2)

with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

Tokens Used: 40
	Prompt Tokens: 4
	Completion Tokens: 36
Successful Requests: 1
Total Cost (USD): $7.8e-05


# Few Shot Learning

This approach involves using the **FewShotPromptTemplate** class, which takes in a **PromptTemplate** and a list of a few shot examples. The class formats the prompt template with a few shot examples, which helps the language model generate a better response. We can streamline this process by utilizing LangChain's FewShotPromptTemplate to structure the approach:

In [28]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

examples = [
            {
            "query":"What's the weather like?",
            "answer":"It's raining cats and dogs today. better bring an umbrella!"
            },
            {
            "query":"how old are you?",
            "answer":"Age is just a number, but i am timeless."
            }
]

example_template = """
                   User: {query}
                   AI : {answer}
                   """

example_prompt = PromptTemplate(
                    input_variables = ["query", "answer"],
                    template = example_template
                    )

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [29]:
from langchain import LLMChain
llm = OpenAI(model = 'gpt-3.5-turbo-instruct', temperature=0)

chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)
print(chain.run("What's the meaning of life?"))

42, according to Douglas Adams. But personally, I think it's to enjoy the little things and make the most out of every moment.


# Emergent abilities, Scaling laws, and hallucinations

Another aspect of LLMs is their emergent abilities, which arise as a result of extensive pre-training on vast datasets. These capabilities are not explicitly programmed but emerge as the model discerns patterns within the data. LangChain models capitalize on these emergent abilities by working with various types of models, such as chat models and text embedding models. This allows LLMs to perform diverse tasks, from answering questions to generating text and offering recommendations.

Lastly, scaling laws describe the relationship between model size, training data, and performance. Generally, as the model size and training data volume increase, so does the model's performance. However, this improvement is subject to diminishing returns and may not follow a linear pattern. It is essential to weigh the trade-off between model size, training data, performance, and resources spent on training when selecting and fine-tuning LLMs for specific tasks.

While Large Language Models boast remarkable capabilities but are not without shortcomings, one notable limitation is the occurrence of hallucinations, in which these models produce text that appears plausible on the surface but is actually factually incorrect or unrelated to the given input. Additionally, LLMs may exhibit biases originating from their training data, resulting in outputs that can perpetuate stereotypes or generate undesired outcomes.

# Examples with Easy Prompts: Text Summarization, Text Translation, and Question Answering

Creating a Question-Answering Prompt Template

In [30]:
!pip install -q huggingface_hub

In [31]:
template = """Question: {question}

Answer: """
prompt = PromptTemplate(
    template=template,
    input_variables=['question']
)

# user question
question = "What is the capital city of France?"

# Next, we will use the Hugging Face model google/flan-t5-largeCopy to answer the question. 
# The HuggingfaceHubCopy class will connect to Hugging Face’s inference API and load the specified model.

In [35]:
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
hub_llm = HuggingFaceHub(
        repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about the capital of France
print(llm_chain.run(question))

paris


Asking Multiple Questions

In [36]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
print( res )

generations=[[Generation(text='paris')], [Generation(text='giraffe')], [Generation(text='nitrogen')], [Generation(text='yellow')]] llm_output=None run=[RunInfo(run_id=UUID('706d225d-a9d2-4598-8e9b-5857fb1333f6')), RunInfo(run_id=UUID('c2ffcf3d-01e2-4935-a11c-5ff6af92f9a1')), RunInfo(run_id=UUID('1bb30e12-7905-4c18-ad96-0d492f1aa388')), RunInfo(run_id=UUID('a42c732b-8108-4da2-859e-2bfdc4a5ce21'))]


In [37]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
		"What color is a ripe banana?\n"
)
llm_chain.run(qs_str)

"\n1. The capital city of France is Paris.\n2. The largest mammal on Earth is the blue whale.\n3. The gas most abundant in Earth's atmosphere is nitrogen.\n4. A ripe banana is typically yellow."

Text Summarization

In [43]:
# using turbo instruct

llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

text = """LangChain provides many modules that can be used to build language model applications. 
Modules can be combined to create more complex applications, or be used individually for simple applications. 
The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. 
For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."""

summarized_text = summarization_chain.predict(text=text)

print(summarized_text)



LangChain offers various modules for building language model applications, which can be used individually or combined to create more complex applications, with the most basic being calling an LLM on input to generate a company name based on its product.


In [45]:
# using turbo

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

text = """LangChain provides many modules that can be used to build language model applications. 
Modules can be combined to create more complex applications, or be used individually for simple applications. 
The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. 
For this purpose, let’s pretend we are building a service that generates a company name based on what the company makes."""

summarized_text = summarization_chain.predict(text=text)

print(summarized_text)

LangChain offers various modules for building language model applications, which can be used individually or combined to create more complex applications, with the basic building block being calling an LLM on input, as demonstrated in a simple example of generating a company name based on its product.


Text translation

In [47]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"
translation_prompt = PromptTemplate(input_variables=["source_language", "target_language", "text"], template=translation_template)
translation_chain = LLMChain(llm=llm, prompt=translation_prompt)

source_language = "English"
target_language = "French"
text = "Your text here"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)

print(translated_text)

Votre texte ici
